<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado el 2018-1 y 2018-2.</font>
</p>

# Tabla de contenidos
1. [Multiherencia](#Multiherencia)
    1. [Multiherencia y el problema del diamante](#Multiherencia-y-el-problema-del-diamante)
        1. [Solución](#Solución)
        2. [El método `__mro__`](#El-método-__mro__)
    2. [Ejemplo multiherencia](#Ejemplo-multiherencia)
        1. [`*args` y `**kwargs`](#Forma-mejorada:-*args-y-**kwargs)

## Multiherencia

Tal como es posible que una subclase herede datos y comportamiento de una superclase, también es posible heredar
de más de una clase a la vez. 

Supongamos una clase `Academico`. Un académico posee dos roles: investigador y docente, por lo tanto en nuestro modelo reflejaremos este hecho haciendo que `Academico` herede de las clases ya existentes `Investigador` y `Docente`. Esto lo implementaríamos de la siguiente manera.

In [1]:
class Investigador:
    def __init__(self, area):
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 3
        

#Aquí decimos que Academico hereda tanto de Docente como de Investigador
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Esto no es del todo correcto, pero lo explicamos más abajo.
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina

p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


En este ejemplo, como la clase `Academico` hereda tanto de `Docente` como de `Investigador`, parece natural llamar a ambos métodos `__init()__` de cada una de sus clases superiores. Sin embargo esto, puede provocar problemas en modelos más complejos.

### Multiherencia y el problema del diamante

El siguiente ejemplo muestra lo que ocurre en un contexto de multiherencia si es que cada subclase llama directamente a inicializar a todas sus superclases. La figura muestra una jerarquía de clases en que una `SubclaseA` hereda de dos superclases donde, a su vez, ambas derivan de una misma `ClaseB`. A este modelo que se forma le llamamos _jerarquía de **diamante**_.

![Diamante](img/diamante.png)

Tenemos una jerarquía de diamante cada vez que tenemos más de un _camino_ en la jerarquía desde la clase inferior a una clase superior.

Veamos qué ocurre cuando llamamos al método `llamar()` en ambas superclases desde la clase `SubClaseA`.

In [2]:
class ClaseB:
    
    num_llamadas_B = 0
    
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    
    num_llamadas_izq = 0
    
    def llamar(self):
        print("Estoy en Subclase Izquierda")
        ClaseB.llamar(self)
        print("Llamando método en Subclase Izquierda")
        self.num_llamadas_izq += 1

        
class SubClaseDerecha(ClaseB):
    
    num_llamadas_der = 0
    
    def llamar(self):
        print("Estoy en Subclase Derecha")
        ClaseB.llamar(self)
        print("Llamando método en Subclase Derecha")
        self.num_llamadas_der += 1

        
class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    
    num_llamadas_subA = 0
    
    def llamar(self):
        print("Estoy en Subclase A")        
        SubClaseIzquierda.llamar(self)
        SubClaseDerecha.llamar(self)
        print("Llamando método en Subclase A")
        self.num_llamadas_subA += 1


s = SubClaseA()
s.llamar()
print()
print(f"Llamadas en Subclase A: {s.num_llamadas_subA}")
print(f"Llamadas en Subclase Izquierda: {s.num_llamadas_izq}")
print(f"Llamadas en Subclase Derecha: {s.num_llamadas_der}")
print(f"Llamadas en Clase B: {s.num_llamadas_B}")

Estoy en Subclase A
Estoy en Subclase Izquierda
Llamando método en Clase B
Llamando método en Subclase Izquierda
Estoy en Subclase Derecha
Llamando método en Clase B
Llamando método en Subclase Derecha
Llamando método en Subclase A

Llamadas en Subclase A: 1
Llamadas en Subclase Izquierda: 1
Llamadas en Subclase Derecha: 1
Llamadas en Clase B: 2


Podemos apreciar que el método `llamar` la clase de más arriba en la jerarquía (`ClaseB`) fue llamada dos veces. Luego de cada ejecución de `llamar`, la secuencia de invocaciones sube por la jerarquía hasta el método correspondiente en `ClaseB`.

La estructura de jerarquía en forma de diamante ocurre **siempre** que tengamos una clase que hereda de dos clases, aun cuando no tengamos una tercera superclase explícita. ¿Por qué? Porque en Python (y en varios lenguajes OOP), existe una clase [`object`](https://docs.python.org/3.6/library/functions.html#object) de la cual heredan **todas** las clases que creamos. 

En particular, cuando se utiliza multiherencia, el esquema se ve de la siguiente forma:

![Diamante2](img/diamante_2.png)

De esta manera, si, estando en un objeto de `SubClase`, llamamos al método `__init__` tanto de `ClaseA` como de `ClaseB`, estaríamos inicializando dos veces la clase `object`. Eso es precisamente lo que ocurre en el ejemplo de clase `Academico`, la cual llama explícitamente al inicializador de `Investigador` y de `Docente`, lo que tiene como consecuencia que la clase `object` se inicializa dos veces.


#### Solución

La solución es que cada clase debe preocuparse de llamar a inicializar a la clase que la "precede" en el orden del esquema de la multiherencia. En Python, cada jerarquía posee un orden predefinido por la construcción de la jerarquía. El orden de las clases va **de izquierda a derecha** dentro de la lista de superclases desde donde hereda la subclase. 

En el siguiente ejemplo, basado en la primera jerarquía de diamante que presentamos, sólo debemos preocuparnos de hacer una llamada a `super()`. Python se encargará de que la llamada corresponda a la clase que respeta el orden en la multiherencia.

In [3]:
class ClaseB:
    
    num_llamadas_B = 0
    
    def llamar(self):
        print("Llamando método en Clase B")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    
    num_llamadas_izq = 0
    
    def llamar(self):
        print("Estoy en Subclase Izquierda")
        super().llamar()
        print("Llamando método en Subclase Izquierda")
        self.num_llamadas_izq += 1

        
class SubClaseDerecha(ClaseB):
    
    num_llamadas_der = 0
    
    def llamar(self):
        print("Estoy en Subclase Derecha")
        super().llamar()
        print("Llamando método en Subclase Derecha")
        self.num_llamadas_der += 1

        
class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    
    num_llamadas_subA = 0
    
    def llamar(self):
        print("Estoy en Subclase A")
        super().llamar()
        print("Llamando método en Subclase A")
        self.num_llamadas_subA += 1


s = SubClaseA()
s.llamar()
print()
print(f"Llamadas en Subclase A: {s.num_llamadas_subA}")
print(f"Llamadas en Subclase Izquierda: {s.num_llamadas_izq}")
print(f"Llamadas en Subclase Derecha: {s.num_llamadas_der}")
print(f"Llamadas en Clase B: {s.num_llamadas_B}")

Estoy en Subclase A
Estoy en Subclase Izquierda
Estoy en Subclase Derecha
Llamando método en Clase B
Llamando método en Subclase Derecha
Llamando método en Subclase Izquierda
Llamando método en Subclase A

Llamadas en Subclase A: 1
Llamadas en Subclase Izquierda: 1
Llamadas en Subclase Derecha: 1
Llamadas en Clase B: 1


Podemos ver que esta vez estamos llamando solamente una vez al método `llamar` de `ClaseB`. Es más, si bien el diamante sigue existiendo, esta vez bastó ejecutar solo una vez el método `llamar` de `SubclaseA`, y conseguimos ejecutar el método `llamar` de todas las clases, y solamente una vez cada uno. 

Si ponemos atención a la secuencia de llamados, pareciera que hay un _orden_ impuesto entre las clases que componen el diamante. Este orden es: primero `SubClaseA`, luego `SubClaseIzquierda`, a continuación `SubClaseDerecha`, y finalmente `ClaseB`.

#### El método `__mro__` 

La solución para determinar en qué orden se ejecutan los métodos en un esquema de multiherencia, se estableció mediante un algoritmo llamado [**C3**](https://www.python.org/download/releases/2.3/mro/) que permite calcular un orden lineal entre las clases que participan del esquema. Este algoritmo puede ser ejecutado por todas las clases de Python usando el método predefinido **`__mro__`**, cuyo nombre viene de _method resolution order_. Este método nos muestra el orden en la jerarquía de clases a partir de la clase actual. Es útil para casos de multiherencia complejos.

In [4]:
SubClaseA.__mro__

(__main__.SubClaseA,
 __main__.SubClaseIzquierda,
 __main__.SubClaseDerecha,
 __main__.ClaseB,
 object)

Notemos que el resultado de `__mro__` depende de la clase a la cual se aplica. Si lo aplicamos a `SubClaseIzquierda`, entonces su MRO no incluye a `SubClaseDerecha` pues ésta no es parte de su jerarquía.

In [5]:
SubClaseIzquierda.__mro__

(__main__.SubClaseIzquierda, __main__.ClaseB, object)

Ahora bien, **no toda estructura de multiherencia está permitida**. No es tan difícil armar una jerarquía en que no sea posible armar un MRO consistente para todas las clases. Por ejemplo:

In [6]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(Y, X):
     def call_me(self):
        print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

En este ejemplo, tanto `A` como `B` heredan de `X` e `Y` pero en distinto orden. Esto no es un problema hasta que se define la clase `F` que hereda de `A` y de `B`. En este momento Python prohibe la creación de la clase pues no puede determinar un MRO consistente para llegar a `X` e `Y`.

Para este caso, basta modificar `A` y `B` para que hereden de la misma manera, y ahora sí se puede determinar un MRO.

In [7]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(X, Y):
     def call_me(self):
        print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")
        
print(F.__mro__)

(<class '__main__.F'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.X'>, <class '__main__.Y'>, <class 'object'>)


### Ejemplo multiherencia

Recordemos el ejemplo del académico que presentamos al inicio del tema de multiherencia. Si bien el ejemplo parece funcionar bien, ahora sabemos que si llamamos por separado a cada inicializador de las superclases, estamos llamando dos veces al inicializador de `object`, y no queremos que esto ocurra.

In [8]:
class Investigador:
    def __init__(self, area):
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 3
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Reemplacemos esto por un super().__init__(...)
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina

p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


La solución, tal como lo hicimos con el método `llamar()` de la sección anterior, debería ser reemplazar las inicializaciones explícitas de `Investigador` y `Docente`, por un único llamado a `super().__init__()`, y así dejamos que MRO haga su trabajo.

Sin embargo, tenemos una situación levemente distinta. ¿Qué argumentos le entregamos a `super().__init__()`? Si le entregamos solamente los de `Investigador` o solamente los de `Docente`, uno de los inicializadores se quedaría sin argumentos.

In [9]:
class Investigador:
    def __init__(self, area):
        print("Inicializando investigador")
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento):
        print("Inicializando docente")
        self.departamento = departamento
        self.num_cursos = 3
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Solo un llamado, pero solo con un argumento
        super().__init__(departamento)
        self.nombre = nombre
        self.oficina = oficina

print(Academico.__mro__)
p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)
Inicializando docente
Emilia Donoso


AttributeError: 'Academico' object has no attribute 'area'

Si bien, hay un MRO definido para `Academico`, cuando solo entregamos los argumentos de un inicializador, solo se ejecuta el `__init()__` de `Docente`, y no el de `Investigador`, por lo tanto nuestro `Academico` se queda sin su atributo `area`. 

Entonces una mejor solución sería entregarle a `super().__init()__` _todos_ los argumentos.

In [10]:
class Investigador:
    def __init__(self, area):
        print("Inicializando investigador")
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento):
        print("Inicializando docente")
        self.departamento = departamento
        self.num_cursos = 3
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Solo un llamado, con todos los argumentos que tenemos
        super().__init__(departamento, area_investigacion)
        self.nombre = nombre
        self.oficina = oficina

print(Academico.__mro__)
p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)


TypeError: __init__() takes 2 positional arguments but 3 were given

Pero esto tampoco sirve, porque cada `__init__()` recibe solamente dos argumentos y le estamos entregando tres (no olvidemos al `self`). El ejemplo inicial con el método `llamar()` era muy ~~tramposo~~ sencillo porque `llamar()` no tenía argumentos. Estamos en un dilema.

####  `*args` y `**kwargs` al rescate

El dilema que tenemos se produce porque, aunque entreguemos todos los argumentos a `super().__init__()`, ninguno de los inicializadores sabe cuáles argumentos son para él, y cuáles para otro inicializador. Pero Python provee una solución.

`**kwargs` es una _secuencia de argumentos de largo variable_, donde cada elemento de la lista tiene asociado un **keyword**. El `**` mapea los elementos contenidos en el diccionario `kwargs` y los pasa a la función como _argumentos no posicionales_. Esto significa que los argumentos no se asignan a la función por su posición en el orden en que se entregan (como es lo habitual) sino por su _keyword_ asociado. De ahí el nombre _kwargs_ o _keyword arguments_. El `**kwargs` puede ser usado para enviar una cantidad variable de argumentos.

Para entender mejor el funcionamiento, observemos el siguiente ejemplo:

In [4]:
def funcion(arg1, arg2, arg3):
    print(f"arg1: {arg1}")
    print(f"arg2: {arg2}")
    print(f"arg3: {arg3}")

kwargs = {"arg3": 3, "arg2": "two"}
funcion(1, **kwargs)

arg1: 1
arg2: two
arg3: 3


`*args` es un mecanismo similar. `*args`, es una _lista de argumentos de largo variable_, pero sin _keywords_ asociados. El operador `*` _desempaqueta_ el contenido de `args` y los pasa a la función como _argumentos posicionales_. La función asigna valores a sus argumentos a partir del orden que trae esta lista.

El siguiente ejemplo ilustra su uso:

In [12]:
def funcion(arg1, arg2, arg3):
    print(f"arg1: {arg1}")
    print(f"arg2: {arg2}")
    print(f"arg3: {arg3}")

args = ["two", 3]
funcion(1, *args)

arg1: 1
arg2: two
arg3: 3


De esta manera, `funcion` recibe su valor para `arg1` de manera tradicional, y a continuación recibe dos valores más donde el _primero_ lo asigna a `arg2` y el _segundo_ a `arg3`, ya que esta vez los argumentos son _posicionales_.

Pero no solo podemos aprovechar `*args` y `**kwargs` para _entregar_ argumentos, sino también para _recibirlos_.

In [13]:
def funcion(arg1, *argv):
    print(f"primer argumento normal: {arg1}")
    for arg in argv:
        print(f"siguiente argumento de *argv: {arg}")

funcion('hola','como','va','todo')
funcion('bien,', 'porque', 'entiendo', '*args', 'y', '**kwargs')
funcion('ok')

primer argumento normal: hola
siguiente argumento de *argv: como
siguiente argumento de *argv: va
siguiente argumento de *argv: todo
primer argumento normal: bien,
siguiente argumento de *argv: porque
siguiente argumento de *argv: entiendo
siguiente argumento de *argv: *args
siguiente argumento de *argv: y
siguiente argumento de *argv: **kwargs
primer argumento normal: ok


Aquí `funcion` recibe dos argumentos, pero el segundo es una lista empaquetada de argumentos. Esto nos permite que en cada llamado entreguemos una cantidad distinta de argumentos y `funcion` recorre su lista `argv` para acceder a cada uno de ellos. Es posible también, que el valor de `*argv` esté vacío, sin embargo debe recibir al menos _un_ argumento `arg1` de manera normal.

Más ejemplos:

In [14]:
def funcion(a=0, b=0):
    return a + b

# Usando solo un valor posicional y el resto usa los argumentos por defecto
valores = (1, )
print(funcion(*valores))

# Usando todos los argumentos posicionales definidos en la lista
valores = (1, 2)
print(funcion(*valores))

1
3


In [15]:
def funcion(a=0, b=0):
    return a + b

# La función necesita dos argumentos, por lo tanto el exceso de argumentos posicionales genera un error.
valores = (1, 2, 3)
print(funcion(*valores))

TypeError: funcion() takes from 0 to 2 positional arguments but 3 were given

Este ejemplo permite no solo entregar una cantidad arbitraria de argumentos, sino también permite asignarle un nombre a cada nuevo argumento.

In [16]:
def funcion(arg1, **argumentos):
    print(f"primer argumento normal: {arg1}")
    for arg in argumentos:
        print(f"siguiente argumento de **argumentos: {arg} --> {argumentos[arg]}")

funcion('hola', Uno='como', Dos='va', Tres='todo')
funcion('bien,', Dos='porque', otros='entiendo', nombres='*args', para='y', argumentos='**kwargs')
funcion('ok')

primer argumento normal: hola
siguiente argumento de **argumentos: Uno --> como
siguiente argumento de **argumentos: Dos --> va
siguiente argumento de **argumentos: Tres --> todo
primer argumento normal: bien,
siguiente argumento de **argumentos: Dos --> porque
siguiente argumento de **argumentos: otros --> entiendo
siguiente argumento de **argumentos: nombres --> *args
siguiente argumento de **argumentos: para --> y
siguiente argumento de **argumentos: argumentos --> **kwargs
primer argumento normal: ok


También es posible usar conjuntamente `*args` y `**kwargs`. En ese caso, debemos tener cuidado que en una función el orden debe ser: primero argumentos posicionales (normales), luego `*args` y finalmente `**kwargs`. Por ejemplo:


In [17]:
def funcion(arg_fijo, *args_posicionales, **args_con_nombre):
    print(f"primer argumento normal: {arg_fijo}")
    for arg in args_posicionales:
        print(f"siguiente argumento posicional: {arg}")
    
    for arg in args_con_nombre:
        print(f"siguiente argumento de **argumentos: {arg} --> {args_con_nombre[arg]}")

funcion('hola', 'como', 'va', Tres='todo')
funcion('bien,', 'porque', 'entiendo', nombres='*args', para='y', argumentos='**kwargs')
funcion('ok')

primer argumento normal: hola
siguiente argumento posicional: como
siguiente argumento posicional: va
siguiente argumento de **argumentos: Tres --> todo
primer argumento normal: bien,
siguiente argumento posicional: porque
siguiente argumento posicional: entiendo
siguiente argumento de **argumentos: nombres --> *args
siguiente argumento de **argumentos: para --> y
siguiente argumento de **argumentos: argumentos --> **kwargs
primer argumento normal: ok


Si bien nos hemos referido todo el tiempo a `*args` y `**kwargs`, los _operadores_ reales son `*` y `**` que indican respectivamente _desempaquetamiento_ de secuencias iterables (listas, tuplas), y de diccionarios. Los nombres que usamos `args` y `kwargs` son convenciones.

#### Ejemplo arreglado

Ahora que conocemos el uso de `*args` y `**kwargs`, podemos aplicarlos para, finalmente, implementar correctamente la inicialización en el ejemplo del académico. Recordemos el código que queremos mejorar:

In [18]:
class Investigador:
    def __init__(self, area):
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 3
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Queremos reemplazar esto por un super().__init__(...), pero no sabemos qué argumentos usar
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina

p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


Deseamos reemplazar los llamados a ambos inicializadores, por una única invocación `super().__init__()`, pero no sabemos qué argumentos entregar.

Aprovecharemos el hecho que `**kwargs` nos permite entregar un diccionario de argumentos.

In [9]:
class Investigador:
    def __init__(self, area, **kwargs):
        print(f"init Investigador con area {area} y kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.area = area
        self.num_publicaciones = 0
        
class Docente:
    def __init__(self, departamento, **kwargs):
        print(f"init Docente con depto {departamento} y kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.departamento = departamento
        self.num_cursos = 3
        
class Academico(Docente, Investigador):
    def __init__(self, nombre, oficina, **kwargs):
        print(f"init Academico con nombre {nombre}, oficina {oficina}, kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.nombre = nombre
        self.oficina = oficina

print(Academico.__mro__)
p1 = Academico("Emilia Donoso", oficina="O5", area="Inteligencia de Máquina", departamento="Ciencia De La Computación")
print("--------")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)
init Academico con nombre Emilia Donoso, oficina O5, kwargs:{'area': 'Inteligencia de Máquina', 'departamento': 'Ciencia De La Computación'}
init Docente con depto Ciencia De La Computación y kwargs:{'area': 'Inteligencia de Máquina'}
init Investigador con area Inteligencia de Máquina y kwargs:{}
--------
Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


Este ejemplo permite, finalmente, llamar **una sola vez** y de manera correcta a todos los inicializadores de las clases de una jerarquía con multiherencia. El diccionario `**kwargs` contiene los argumentos identificados por su nombre, de manera que en cada llamado a un inicializador, éste extrae los _keywords_ que correspondan a algún nombre de los argumentos que espera, y el resto permanecen en el `**kwargs` y son pasados como argumento a la siguiente clase en la jerarquía. De esta manera, cada inicializador _consume_ del `**kwargs` lo que necesita. Notemos que en la última clase de la jerarquía, antes de llamar a `object`, todos los _keywords_ en `**kwargs` han sido consumidos, lo que está bien porque el inicializador de `object` no recibe argumentos.

Una adecuada comprensión de los llamados a métodos/funciones usando lista variables de argumentos es muy práctica y flexible. Lo estaremos ocupando en numerosas ocasiones.
